# Создание Телеграм-ботов. Inline-боты.

Сегодня мы с вами выясним, что такое встраиваемые боты, и как их писать. Параллельно поговорим о регулярных выражениях.

---

Ссылки:
* Описание всех методов и классов Telegram Bot API: https://core.telegram.org/bots/api
* Краткая документация по pyTelegramBotAPI: https://pypi.org/project/pyTelegramBotAPI/
* Справочник по Telegram Bot API: https://tlgrm.ru/docs/bots/api
* Исходный код библиотеки pyTelegramBotAPI: https://github.com/eternnoir/pyTelegramBotAPI
* Гайд по созданию Телеграм-ботов №1: https://m.habr.com/ru/post/350648/
* Гайд по созданию Телеграм-ботов №2: https://mastergroosha.github.io/telegram-tutorial/

# Что такое inline-бот?

Инлайн боты используются, чтобы отправлять какие-то сообщения в ваш чат с другим пользователем.  
В любом чате вы набираете имя бота и через пробел команду для него. В ответ бот присылает вам различные варианты сообщений.  

Такие боты, например, могут использоваться, чтобы отправлять ссылки на смешные видео из YouTube.  

<table><tr><td><img src="инлайн1.jpg" style="height:400px;"/></td> <td><img src="инлайн2.jpg" style="height:400px;"/></td> </tr></table>

# Регистрируем inline-возможности в @BotFather

Инлайн-функции можно дописать к любому из тех ботов, которых мы писали ранее. И вообще к любому боту. Но для этого нужно активировать инлайн-режим в @BotFather.

Для этого:
* Открыть диалог с @BotFather, прописать команду `/setinline` и следовать инструкциям.
* Ввести подсказку, которая будет выводиться, когда пользователь напишет имя вашего бота.

Теперь инлайн-функции доступны для вашего бота.

# Хэндлер

Инлайн-команды отлавливаются хэндлером `@bot.inline_handler(filters)`. Важно понимать, что этот хэндлер отлавливает не уже отправленное пользователем сообщение, а ввод. Каждый раз, когда пользователь делает паузу в наборе текста - Телеграм считает это потенциальной командой и отправляет боту.  

Из фильтров здесь есть только `func`. Функция, которую передают в хэндлер должна принимать не `message`, а `query`. То же самое касается и функции, которая вызывается хэндлером.

In [ ]:
@bot.inline_handler(func=lambda query: True)
def some_action(query):
    pass

# Query

Давайте вкратце разберемся со структурой `query`:

In [ ]:
{
    'id': '2273168582891559192',              # id запроса
    'from_user': {                            # все про пользователя
        'id': 529263304,
        'is_bot': False,
        'first_name': 'Михаил',
        'username': 'meshanya',
        'last_name': 'Михайлов',
        'language_code': 'ru',
        'can_join_groups': None,
        'can_read_all_group_messages': None,
        'supports_inline_queries': None},
    'location': None,
    'query': '2 10',                          # текст команды (без имени бота)
    'offset': ''                              # про offset читайте ниже
}

# Как отвечать на inline-запросы?

Все ответы делаются через `bot.answer_inline_query(inline_query_id=query.id, results=[ans1, ans2, ans3])`.  

`results` - это список с возможными вариантами ответа на запрос пользователя.  
Например, для бота с видео из YouTube - это были различные видео.

У `answer_inline_query` есть еще несколько аргументов:  
* `cache_time` - время кэширования ответа в секундах (по умолчанию - 300). Если пользователь в течение этого времени сделает запрос, который уже задавал ранее, то ему покажут старый ответ, сохраненный на серверах Телеграм. Если у вашего бота ответ на тот же самый вопрос может меняться со временем - не устанавливайте это значение большим.
* `is_personal` - если установить `True`, то результат будет кэшироваться только для пользователя, который сделал запрос.
* `next_offset` - читайте про `offset` далее.
* `switch_pm_text` - текст для специальной кнопки. Про эту кнопку - ниже.
* `switch_pm_parameter` - параметр для специальной кнопки. 


## Как задавать варианты ответа?

Аргумент `results` в `bot.answer_inline_query` задается переменными `InlineQueryResult`. Всего есть 19 типов различных ответов. Разберу основные:  

### InlineQueryResultArticle
Задает текстовый ответ. Подсказки выглядят так же, как на картинках с YouTube ботом. К каждой подсказке добавляется картика, заголовок и краткое описание.

In [ ]:
ans = types.InlineQueryResultArticle(
                id='1', title="Заголовок",
                description="Описание",
                input_message_content=types.InputTextMessageContent(message_text="Сообщение"),
                thumb_url="Ссылка на картинку", thumb_width=48, thumb_height=48
        )

* `id` - уникальный идентификатор ответа.
* `title` - заголовок подсказки.
* `description` - описание ответа на подсказке.
* `input_message_content` - тип контента, который будет отправлен пользователю. Помимо текстового сообщения можно отправить местоположение и контакт.
* `thumb_url` - ссылка на картику, которая будет показываться на превью.
* `thumb_width` - ширина картинки.
* `thumb_height` - высота картинки.

### InlineQueryResultPhoto

Задает ответ-изображение. Подсказки - галерея с фотографиями. К ответу можно добавить подпись и кнопки (`InlineKeyboardMarkup`). Изображение задается **ссылкой**.

In [ ]:
ans = types.InlineQueryResultPhoto(id='1', 
                                   photo_url=plus_icon, 
                                   thumb_url=minus_icon,
                                   photo_width=100, 
                                   photo_height=100, 
                                   caption='Подпись',
                                   reply_markup=markup)

Есть второй вариант использования функции. Вместо `photo_url` можно указать `input_message_content`. Тогда подсказки будут галереей фотографий, а ответ - как в `InlineQueryResultArticle`.  

---

Есть еще ряд функций, аналогичных `InlineQueryResultPhoto`: `InlineQueryResultAudio`, `InlineQueryResultVideo` и другие. Они соответствуют другим типам медиа.  


### InlineQueryResultCachedPhoto

Делает то же самое, что и `InlineQueryResultPhoto`, но задает изображение не по ссылке, а по `file_id`. О том, как получить `file_id` для медиафайла - поговорим в следующих лекциях.  

---

Соответственно, есть еще ряд функций, аналогичных `InlineQueryResultCachedPhoto`: `InlineQueryResultCachedAudio`, `InlineQueryResultCachedVideo` и другие.

# О кнопках и редактировании сообщений

Инлайновые сообщения точно так же, как и обычные, можно редактировать. Единственная разница, что для их идентификации нужна не связка `chat_id` + `message_id`, а параметр `inline_message_id`.  

К инлайновым сообщениям точно так же, как и к обычным, можно прикреплять кнопки (`InlineKeyboardMarkup`).  

Здесь показан пример обработчика коллбэков от Callback-кнопок, который обрабатывает как кнопки от обычных сообщений, так и от инлайновых:

In [ ]:
@bot.callback_query_handler(func=lambda call: call.data == "test")
def callback_inline(call):
    # Если сообщение из чата с ботом
    if call.message:
        bot.edit_message_text(chat_id=call.message.chat.id, message_id=call.message.message_id, text="Обычное сообщение")
    # Если сообщение из инлайн-режима
    elif call.inline_message_id:
        bot.edit_message_text(inline_message_id=call.inline_message_id, text="Инлайн сообщение")


# Switch-кнопки

На прошлой лекции мы говорили о кнопках, которые прикрепляются к сообщению. Один из возможных типов таких кнопок - switch-кнопки. Этот тип кнопок нужен для обучения пользователей инлайн-режиму бота.

При нажатии на эту кнопку, выбранному пользователю отправится сообщение с ником бота и какой-то инлайн-командой.  

Создается switch-кнопка так:  
`switch_button = types.InlineKeyboardButton(text="Нажми меня", switch_inline_query='special_inline_command')`  

Здесь поле `switch_inline_query` - это и есть та команда, которую мы хотим отправить в инлайновом режиме.  
Этот параметр можно задать и пустой командой.  

Напомним, что кнопки, которые прикрепляются к сообщению задаются так:

In [ ]:
keyboard = types.InlineKeyboardMarkup()
switch_button = types.InlineKeyboardButton(text="Нажми меня", switch_inline_query='special_inline_command')
keyboard.add(switch_button)
bot.send_message(message.chat.id, "Привет! Нажми на кнопку и узнай, как работает инлайн-команда.", reply_markup=keyboard)

# switch_pm_text и switch_pm_parameter

Продолжая тему switch-кнопок - обсудим аргументы `switch_pm_text` и `switch_pm_parameter` в методе, который задает список подсказок, - `bot.answer_inline_query()`.  

Если задать `switch_pm_text` и `switch_pm_parameter` какими-то строками, то над списком подсказок бота появится кнопка с текстом из `switch_pm_text`. При ее нажатии, вы автоматически перейдете в чат с ботом, и боту будет отправлена команда `\start <строка из switch_pm_parameter>`. Пусть вас не смущает, что в чате видно только `/start`. На деле, в поле `message.text` лежит то, что нужно.  

Если после этого бот отправляет в чат с вами switch-кнопку - вас автоматически перебросит в исходный чат.

---

*Зачем нужен такой функционал?*  

*Например, в боте, который рекомендует видео из YouTube так можно реализовать авторизацию.  
Когда пользователь начинает делать инлайн-запрос - он видит предложение авторизоваться, чтобы улучшить релевантность подсказок. При нажатии на кнопку `авторизоваться` - его перекинет в чат с ботом, куда бот скинет ссылку на авторизацию (чтобы все участники чата, куда вводился инлайн-запрос, этого не видели).*  

*Как только вход на YouTube будет выполнен - боту будет достаточно просто отправить switch-кнопку в чат, чтобы пользователя обратно перекинуло в исходный чат.*

# Offset

В методе, который задает список подсказок - `bot.answer_inline_query()` был еще один непонятный аргумент - `next_offset` (англ. "смещение").  

Этот параметр нужен, чтобы реализовать постепенную подгрузку подсказок. Например, пользователь запрашивает определенное видео - бот возвращает 5 подсказок. Если пользователь начнет их проматывать - бот загрузит еще 5, и Телеграм допишет их к уже имеющимся.  

В переменной `query` есть поле `offset` (это **строка**). Изначально поле `offset` - это пустая строка. Если пользователь не изменит запрос, а начнет листать подсказки - то снова сработает хэндлер, но в `query.offset` уже будет записано значение, которое мы передали в `next_offset` (передавать тоже надо **строку**). Это позволяет боту определить что нужна новая порция подсказок.

In [ ]:
@bot.inline_handler(func=lambda query: True)
def show_hints(query):
    offset = int(query.offset) if query.offset else 0
    
    if offset == 20:
        return
    
    res = []
    for i in range(offset, offset + 5):  # Выдаю подсказки порциями по 5
        hint = types.InlineQueryResultArticle(id=i, title="Подсказка №{}".format(i),
                                              description="Очень\nобъемное\nописание\nподсказки.\n",
                                              input_message_content=types.InputTextMessageContent(
                                                message_text="Подсказка №{}".format(i)))
        res.append(hint)
        
    bot.answer_inline_query(query.id, res, cache_time=5, next_offset=str(offset + 5))

# Регулярные выражения

Наконец, обсудим, что такое регулярные выражения.  

В инлайн-ботах очень актуальна следующая задача: нужно отсекать ситуации, когда пользователь задумался и ввел команду не до конца.  
Например, вы писали бота калькулятор, а пользователь ввел только одно число. При таком раскладе ваш бот ничего сделать не может.  

Регулярные выражения как раз нужны, чтобы определять соответствует ли текст или его часть заданным параметрам.  

---

Для работы с регулярками используется модуль `re`. Я пробегусь по его основным функциям.  
Подробнее про регулярные выражения почитайте здесь:
* Основы: https://tproger.ru/translations/regular-expression-python/
* И еще: https://tproger.ru/translations/regular-expression-python/
* Посложнее: https://habr.com/ru/post/349860/

Регулярное выражение - это строка из специальных символов. Она задает правила, по которым одни строки удоблетворяют этому выражению, а другие - нет.

Основные обозначения в регулярных выражениях:
* `.` - Один любой символ, кроме новой строки `\n`.
* `?` - 0 или 1 вхождение шаблона слева
* `+` - 1 и более вхождений шаблона слева
* `*` - 0 и более вхождений шаблона слева
* `\w` - Любая цифра или буква (`\W` — все, кроме буквы или цифры)
* `\d` - Любая цифра `[0-9]` (`\D` — все, кроме цифры)
* `\s` - Любой пробельный символ (\S — любой непробельный символ)
* `\b` - Граница слова
* `[..]` - Один из символов в скобках (`[^..]` — любой символ, кроме тех, что в скобках)
* `\` - Экранирование специальных символов (`\.` означает точку или `\+` — знак «плюс»)
* `^` и `$` - Начало и конец строки соответственно
* `{n,m}` - От n до m вхождений (`{,m}` — от 0 до m)
* `a|b` - Соответствует a или b
* `()` - Группирует выражение и возвращает найденный текст
* `\t`, `\n`, `\r` - Символ табуляции, новой строки и возврата каретки соответственно

 <table style="font-size:14px">
   <tr>
    <th>
     Регулярка
    </th>
    <th>
     Её смысл
    </th>
   </tr>
   <tr>
    <td>
      simple text
    </td>
    <td>
     В точности текст «simple text»
    </td>
   </tr>
   <tr>
    <td>
     <code>\d{5}</code>
    </td>
    <td>
     Последовательности из 5 цифр
     <br/>
     <code>\d</code>
     означает любую цифру
     <br/>
     <code>{5}</code>
     — ровно 5 раз
    </td>
   </tr>
   <tr>
    <td>
     <code>\d\d/\d\d/\d{4}</code>
    </td>
    <td>
     Даты в формате ДД/ММ/ГГГГ
     <br/>
     (и прочие куски, на них похожие, например, 98/76/5432)
    </td>
   </tr>
   <tr>
    <td>
     <code>\b\w{3}\b</code>
    </td>
    <td>
     Слова в точности из трёх букв
     <br/>
     <code>\b</code>
     означает границу слова
     <br/>
     (с одной стороны буква, а с другой — нет)
     <br/>
     <code>\w</code>
     — любая буква,
     <br/>
     <code>{3}</code>
     — ровно три раза
    </td>
   </tr>
   <tr>
    <td>
     <code>[-+]?([1-9]\d*|0)</code>
    </td>
    <td>
     Целое число, например, 7, +17, -42
     <br/>
        (исключены ведущие нули, но допустимы 0, +0, -0)
     <br/>
     <code>[-+]?</code>
     — либо -, либо +, либо пусто
     <br/>
     <code>\d+</code>
     — последовательность из 1 или более цифр
    </td>
   </tr>
   <tr>
    <td>
     <code>/w+@w+.w+</code>
    </td>
    <td>
     Электронная почта
     <br/>
     Если мы считаем, что в ней посередине обязательно должна быть собака,
     <br/>
     а после нее - доменное имя с точкой посередине.
    </td>
   </tr>
 </table>


---

Расскажу вкратце о методах, которые могут вам понадобиться сейчас.

In [1]:
import re  # Подключает модуль для регулярных выражений (РВ)

numbers = re.compile(r'[-+]?([1-9]\d*|0)')     # Создаем объект, который будет проверять на соотоветствие РВ

from_first_symbol = numbers.match('13s12')     # Ищет включение начиная с первого символа (вернет None, если ничего не найдет)
print(from_first_symbol.group())               # Нужно использовать метод group, чтобы перейти непосредственно к вхождению
print(from_first_symbol.start(), 
      from_first_symbol.end())                 # Печатаем индексы начала и конца включения

from_arbitrary_symbol = numbers.search('s-12') # Ищет первое включение, начиная с произвольного символа
print(from_arbitrary_symbol.group())

full_match = numbers.fullmatch('-123')         # Проверяет на полное совпадение
print(full_match.group())

all_matches = numbers.findall('a-123bv-0m43')  # Ищет все включения (вернет пустой список, если не найдет)
print(all_matches)

13
0 2
-12
-123
['123', '0', '43']


In [2]:
import re


numb = re.compile(r'[-+]?([1-9]\d*|0)')

inp = input()
while inp != '-1':
    result = numb.fullmatch(inp)
    
    if result == None:
        print("NO")
    else:
        print(result.group())
    print()
    inp = input()


-123
-123

ш1
NO

0
0

90
90

-1


### Использование регулярных выражений в ботах

Теперь представим, что мы писали бота-калькулятора. То есть нам надо было в ответ на два введенных числа выдавать подсказками результаты их сложения, вычитания, умножения и деления. Тогда нам нужно было отсекать ситуации, когда пользователь вводил что-то отличное от двух чисел.  

Например, сделать это можно было так:  

In [ ]:
@bot.inline_handler(func=lambda query: True)
def show_hints(query):
    numb = re.compile(r'[-+]?([1-9]\d*|0) [-+]?([1-9]\d*|0)')
    result = numb.fullmatch(query.text)
    
    if result == None:
        return
    
    a, b = [int(val) for val in result.group().split()]  # Подходящий нам результат был вида "-12 3".
                                                         # Мы разделили его по пробелу и привели к int.
    
    print("Первое число - это {}".format(a))
    print("Второе число - это {}".format(b))

Кстати, если вы вдруг забыли, регулярные выражения могут быть фильтром в хэндлерах.

Синтаксис такой: `@bot.message_handler(regexp='SOME_REGEXP')`.

Данный хэндлер вызовется, если выражение `re.search(r'SOME_REGEXP', message.text)` что-нибудь найдет.  


In [ ]:
re.search(r'SOME_REGEXP', message.text)

# Эти два выражения делают одно и то же. Актуально и для re.match(), re.fullmatch(), re.findall().

regexp = re.compile(r'SOME_REGEXP')
regexp.search(message.text)

# Пишем бота, который будет подсказывать анекдот.


In [ ]:
# Код в PyCharm-проекте